# Querying from Mongo DataBase

In this notebook, we'll make querys to our Mongo database and create a pandas dataframe. 

-----

**First things first, let's import some things...**

In [5]:
import json
import pandas as pd
from pymongo import MongoClient

<br>

**Creating a Mongo Client**

In [6]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['pbmnet']
curriculos = db.curriculos

<br>

**Querying...**

In [7]:
# curriculos.find_one({"PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.AUTORES.@NOME-PARA-CITACAO":"BONOTTO, E.M."})
d = curriculos.find_one({},{'_id':0,'@NUMERO-IDENTIFICADOR':1,'DADOS-GERAIS.@NOME-COMPLETO': 1,
                            'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
                           'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1})

In [8]:
d

{'@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez'},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': [{'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Weak almost periodic motions, minimality and stability in impulsive semidynamical systems',
      '@ANO-DO-ARTIGO': '2014'}},
    {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'On impulsive semidynamical systems: minimal, recurrent and almost periodic motions',
      '@ANO-DO-ARTIGO': '2014'}},
    {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Negative trajectories in impulsive semidynamical systems',
      '@ANO-DO-ARTIGO': '2015'}},
    {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Convergence for non-autonomous semidynamical systems with impulses',
      '@ANO-DO-ARTIGO': '2019'}}]}}}

In [9]:
d['DADOS-GERAIS']['@NOME-COMPLETO']

'Manuel Francisco Zuloeta Jimenez'

In [107]:
unwind = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@NOME-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@CODIGO-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@PAIS':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@UF':1,
        'DADOS-GERAIS.@PAIS-DE-NACIONALIDADE': 1,
        'DADOS-GERAIS.@NOME-EM-CITACOES-BIBLIOGRAFICAS': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DETALHAMENTO-DO-ARTIGO.@TITULO-DO-PERIODICO-OU-REVISTA':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.PALAVRAS-CHAVE':1
                                       }},
                            {'$unwind': '$PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO'},
                          ]))

In [115]:
def Unest_Titulo_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'].strip()

def Unest_Endereco_Nome(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@NOME-INSTITUICAO-EMPRESA']
    except:
        value = None
    return value
def Unest_Endereco_UF_Insituicao(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@UF']
    except:
        value = None
    return value
def Unest_Endereco_Pais_Insituicao(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@PAIS']
    except:
        value = None
    return value

def Unest_Endereco_Codigo(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@CODIGO-INSTITUICAO-EMPRESA']
    except:
        value = None
    return value

def Unest_Nacionalidade(cell):
    try:
        value = cell['@PAIS-DE-NACIONALIDADE']
    except:
        value = None
    return value

def Unest_Ano_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'].strip()

def Unest_Palavras_Chave(cell):
    try:
        value = cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['PALAVRAS-CHAVE']
    except:
        value = None
    return value

def Unest_Journal(cell):
    try:
        value = cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DETALHAMENTO-DO-ARTIGO']['@TITULO-DO-PERIODICO-OU-REVISTA'].strip()
    except:
        value = None
    return value

In [111]:
unwind[0]

{'_id': ObjectId('5d31b5b9a4ef2aef4f14294d'),
 '@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez',
  '@NOME-EM-CITACOES-BIBLIOGRAFICAS': 'M. Z. Jimenez;M. Z. JIMENEZ;JIMENEZ, M. Z.;JIMENEZ, M.Z.',
  '@PAIS-DE-NACIONALIDADE': 'Peru',
  'ENDERECO': {'ENDERECO-PROFISSIONAL': {'@CODIGO-INSTITUICAO-EMPRESA': '198100000000',
    '@NOME-INSTITUICAO-EMPRESA': 'Universidade Tecnológica Federal do Paraná',
    '@PAIS': 'Brasil',
    '@UF': 'PR'}}},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Weak almost periodic motions, minimality and stability in impulsive semidynamical systems',
     '@ANO-DO-ARTIGO': '2014'},
    'DETALHAMENTO-DO-ARTIGO': {'@TITULO-DO-PERIODICO-OU-REVISTA': 'Journal of Differential Equations (Print)'},
    'PALAVRAS-CHAVE': {'@PALAVRA-CHAVE-1': 'Sistemas semidinâmicos Impulsivos',
     '@PALAVRA-CHAVE-2': 'Weak almost periodic motions

In [112]:
def Separar_Palavras_Chave(cell):
    if cell == None:
        return None
    return '|'.join([i for i in cell.values()])


In [116]:
df = pd.DataFrame(unwind)

df['NOME-COMPLETO']     = df["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df['NOME-INSTITUICAO']  = df["DADOS-GERAIS"].apply(Unest_Endereco_Nome)
df['UF-INSTITUICAO']    = df["DADOS-GERAIS"].apply(Unest_Endereco_UF_Insituicao)
df['PAIS-INSTITUICAO']  = df["DADOS-GERAIS"].apply(Unest_Endereco_Pais_Insituicao)
df['PAIS']              = df["DADOS-GERAIS"].apply(Unest_Nacionalidade)
df['NOME-CITACOES']     = df["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-EM-CITACOES-BIBLIOGRAFICAS'].strip())
df['CODIGO-INSTITUICAO']= df["DADOS-GERAIS"].apply(Unest_Endereco_Codigo)
df['TITULO-DO-ARTIGO']  = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Titulo_Artigo)
df['ANO-DO-ARTIGO']     = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Ano_Artigo)
df['JOURNAL']           = df['PRODUCAO-BIBLIOGRAFICA'].apply(Unest_Journal)
df['PALAVRAS-CHAVE-ARTIGO'] = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Palavras_Chave)
df['PALAVRAS-CHAVE-ARTIGO'] = df['PALAVRAS-CHAVE-ARTIGO'].apply(Separar_Palavras_Chave)

df.drop(labels=['PRODUCAO-BIBLIOGRAFICA','DADOS-GERAIS'],axis=1,inplace=True)

In [118]:
df.to_csv("./views/Autor_Artigo_Dataframe.csv",sep=';',encoding='latin-1')